# Exercise 03 : Aggregations

In [1]:
import pandas as pd
import sqlite3

## • create a connection to the database using the library sqlite3

In [2]:
try:
    connection = sqlite3.connect('../data/checking-logs.sqlite.sqlite')
    print("Successfully connect to the database")
except sqlite3.Error as error:
    print("Error when connecting to the database", error)

Successfully connect to the database


## • get the schema of the table test

In [3]:
schema_test = pd.io.sql.read_sql("PRAGMA table_info(test);", connection, index_col='cid')
schema_test

,name,type,notnull,dflt_value,pk
cid,,,,,
0,index,INTEGER,0,None,0
1,uid,TEXT,0,None,0
2,labname,TEXT,0,None,0
3,first_commit_ts,TIMESTAMP,0,None,0
4,first_view_ts,TIMESTAMP,0,None,0


## • get only the first 10 rows of the table test to check what the table looks like

In [4]:
look_test = pd.io.sql.read_sql('select * from test limit 10', connection, index_col='index')
look_test

,uid,labname,first_commit_ts,first_view_ts
index,,,,
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035


## • find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query

◦ do this by joining the table with the table deadlines

◦ the diﬀerence should be displayed in hours

◦ do not take the lab ’project1’ into account, it has longer deadlines and will be an outlier

◦ the value should be stored in the dataframe df_min with the corresponding uid

In [5]:
pd.io.sql.read_sql("PRAGMA table_info(deadlines);", connection, index_col='cid')

,name,type,notnull,dflt_value,pk
cid,,,,,
0,index,INTEGER,0,None,0
1,labs,TEXT,0,None,0
2,deadlines,INTEGER,0,None,0


In [6]:
pd.io.sql.read_sql('select * from deadlines', connection, index_col='index', parse_dates=['deadlines'])

,labs,deadlines
index,,
0,laba04,2020-04-26 23:59:59
1,laba04s,2020-04-26 23:59:59
2,laba05,2020-05-03 23:59:59
4,laba06,2020-05-24 23:59:59
5,laba06s,2020-05-24 23:59:59
3,project1,2020-05-16 23:59:59


In [7]:
query_df_min = """
SELECT uid,
MIN(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS diff_min_time
FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'
"""
df_min = pd.io.sql.read_sql(query_df_min, connection).rename({'diff_min_time': 'MIN(diff)'}, axis=1)
df_min

,uid,MIN(diff)
0,user_30,-202


## • do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [8]:
query_df_max = """
SELECT uid,
MAX(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS diff_max_time
FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'
"""
df_max = pd.io.sql.read_sql(query_df_max, connection).rename({'diff_max_time': 'MAX(diff)'}, axis=1)
df_max

,uid,MAX(diff)
0,user_25,-2


## • do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [9]:
query_df_avg = """
SELECT AVG(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS diff_avg_time
FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
WHERE NOT test.labname = 'project1'
"""
df_avg = pd.io.sql.read_sql(query_df_avg, connection).rename({'diff_avg_time': 'AVG(diff)'}, axis=1)
df_avg

,AVG(diff)
0,-89.125


## • we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coeﬃcient between the number of pageviews and the diﬀerence

◦ using only one query, create a table with the columns: uid, avg_diﬀ, pageviews

◦ uid is the uids that exist in the test

◦ avg_diﬀ is the average delta between the first commit and the lab deadline per user

◦ pageviews is the number of Newsfeed visits per user

◦ do not take the lab ’project1’ into account

◦ store it to the dataframe views_diﬀ

◦ use the Pandas method corr() to calculate the correlation coeﬃcient between the number of pageviews and the diﬀerence

In [10]:
query_views_diff = """
SELECT diff.uid, avg_diff, pageviews
FROM
(
    SELECT uid, AVG(cast((julianday(test.first_commit_ts) - julianday(datetime(deadlines.deadlines, 'unixepoch'))) * 24 AS INTEGER)) AS avg_diff
    FROM test LEFT JOIN deadlines ON test.labname = deadlines.labs
    WHERE NOT test.labname = 'project1'
    GROUP BY uid
) AS diff LEFT JOIN
(
    SELECT uid, COUNT(*) AS pageviews
    FROM pageviews
    GROUP BY uid
) AS views ON diff.uid = views.uid
"""
views_diff = pd.io.sql.read_sql(query_views_diff, connection)
views_diff

,uid,avg_diff,pageviews
0,user_1,-64.400000,28
1,user_10,-74.800000,89
2,user_14,-159.000000,143
3,user_17,-61.600000,47
4,user_18,-5.666667,3
5,user_19,-98.750000,16
6,user_21,-95.500000,10
7,user_25,-92.600000,179
8,user_28,-86.400000,149
9,user_3,-105.400000,317


In [11]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.279736
pageviews,-0.279736,1.000000


In [12]:
query_views_diff = """
    SELECT ll.uid, ll.avg_diff, vs.pageviews
    FROM
    (SELECT uid, avg(avg_diff) AS avg_diff FROM
        (SELECT uid, cast((julianday(t.first_commit_ts) - julianday(datetime(dl.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS avg_diff
        FROM test t
        LEFT JOIN deadlines dl ON t.labname = dl.labs
        WHERE NOT t.labname = 'project1'
        GROUP BY uid
        ) kk
    GROUP BY uid) ll
    LEFT JOIN
        (SELECT uid, count(*) AS pageviews
        FROM  pageviews
        GROUP BY uid
        ) vs
        ON ll.uid = vs.uid
    """

views_diff = pd.io.sql.read_sql(query_views_diff, connection)
views_diff

,uid,avg_diff,pageviews
0,user_1,-6.0,28
1,user_10,-39.0,89
2,user_14,-200.0,143
3,user_17,-81.0,47
4,user_18,-4.0,3
5,user_19,-148.0,16
6,user_21,-126.0,10
7,user_25,-148.0,179
8,user_28,-98.0,149
9,user_3,-75.0,317


In [13]:
views_diff.corr()

,avg_diff,pageviews
avg_diff,1.000000,-0.062967
pageviews,-0.062967,1.000000


## • close the connection

In [14]:
connection.close()